In [13]:

#requirement
import pandas as pd
vosol = pd.read_json('vosol.json')
sabt=pd.read_json("sabt.json")
et=pd.read_json("et.json")
import json
import numpy
import requests
from time import sleep 

In [14]:
#اضافه کردن سرپرست و سوپروایزر به فاکتور ها
et=et.rename(columns={"operator_name":"creator"})
vosol=vosol.merge(et.loc[0:,["creator","group"]],on="creator")
vosol=vosol.merge(et.loc[0:,["creator","leaders"]],on="creator")
sabt=sabt.merge(et.loc[0:,["creator","leaders"]],on="creator")
sabt=sabt.merge(et.loc[0:,["creator","group"]],on="creator")

In [15]:
#تعداد شماره 
sales_report=et.groupby(["group"])["entries_count"].sum()
sales_report=sales_report.to_frame().reset_index()

In [16]:
#مبلغ وصولی
vosoli_hamkadeh=vosol.groupby(["group"])["price"].sum()/10000000
vosoli_hamkadeh=vosoli_hamkadeh.to_frame().reset_index()
vosoli_hamkadeh=vosoli_hamkadeh.rename(columns={"price":"payed"})
sales_report=sales_report.merge(vosoli_hamkadeh,on="group",how="left").fillna(0)

#مبلغ وصولی کارتابلی
vosoli_kartabli_hamkadeh=vosol[vosol["factor_type"]=="کارتابلی"]
vosoli_kartabli_hamkadeh=vosoli_kartabli_hamkadeh.groupby(["group"])["price"].sum()/10000000
vosoli_kartabli_hamkadeh=vosoli_kartabli_hamkadeh.to_frame().reset_index()
vosoli_kartabli_hamkadeh=vosoli_kartabli_hamkadeh.rename(columns={"price":"kartabli_payed"})
sales_report=sales_report.merge(vosoli_kartabli_hamkadeh,on="group",how="left").fillna(0)

In [17]:
#مبلغ ثبتی
sabt_hamkadeh=sabt.groupby(["group"])["price"].sum()/10000000
sabt_hamkadeh=sabt_hamkadeh.to_frame().reset_index()
sabt_hamkadeh=sabt_hamkadeh.rename(columns={"price":"paied_of_registration"})
sales_report=sales_report.merge(sabt_hamkadeh,on="group",how="left").fillna(0)

#تعداد ثبت
tedad_sabt_hamkadeh=sabt.groupby(["group"])["price"].count()
tedad_sabt_hamkadeh=tedad_sabt_hamkadeh.to_frame().reset_index()
tedad_sabt_hamkadeh=tedad_sabt_hamkadeh.rename(columns={"price":"number_of_registration"})
sales_report=sales_report.merge(tedad_sabt_hamkadeh,on="group",how="left").fillna(0)

#تعداد ثبت کارتابلی
tedad_sabt_kartabli_hamkadeh=sabt[sabt["factor_type"]=="کارتابلی"]
tedad_sabt_kartabli_hamkadeh=tedad_sabt_kartabli_hamkadeh.groupby(["group"])["price"].count()
tedad_sabt_kartabli_hamkadeh=tedad_sabt_kartabli_hamkadeh.to_frame().reset_index()
tedad_sabt_kartabli_hamkadeh=tedad_sabt_kartabli_hamkadeh.rename(columns={"price":"number_of_kartabli_registration"})
sales_report=sales_report.merge(tedad_sabt_kartabli_hamkadeh,on="group",how="left").fillna(0)

#درصد وصولی
sales_report["paid_percent"]=sales_report["payed"] /sales_report["paied_of_registration"]


In [18]:
#تعداد افراد فعال
et=et.fillna(0)
et1=et[et["entries_count"]>0]
number_of_active_saller=et1.groupby(["group"])["creator"].count()
number_of_active_saller=number_of_active_saller.to_frame().reset_index()
sales_report=sales_report.merge(number_of_active_saller,on="group",how="left").fillna(0)

#میانگین مبلغ وصولی
sales_report["av_paied"]=sales_report["payed"] /sales_report["creator"]*1000000



In [19]:
#مدت مکالمه hour
modat_mokalemehamkadeh=et.groupby(["group"])["call_duration"].sum()/3600
modat_mokalemehamkadeh=modat_mokalemehamkadeh.to_frame().reset_index()
sales_report=sales_report.merge(modat_mokalemehamkadeh,on="group",how="left").fillna(0)
#میانگین مدت مکالمه
sales_report["av_call_duration"]=sales_report["call_duration"] /sales_report["creator"]


In [20]:
#پاسخداد 
pasokhdad_hamkadeh=et.groupby(["group"])["answered_count"].sum()
pasokhdad_hamkadeh=pasokhdad_hamkadeh.to_frame().reset_index()
sales_report=sales_report.merge(pasokhdad_hamkadeh,on="group",how="left").fillna(0)

In [21]:
#ضریب وصولی
sales_report["paid_coe"]=sales_report["kartabli_payed"] /sales_report["answered_count"]*1000000


#درصد پاسخداد
sales_report["answered_percent"]=sales_report["answered_count"] /sales_report["entries_count"]

#درصد خرید
sales_report["bought_percent"]=sales_report["number_of_kartabli_registration"] /sales_report["answered_count"]



In [22]:

# n=0
# while (n<4):
#     list_id=requests.get("http://192.168.24.134:8000/api/report/sales/group-list")
#     if list_id.status_code==200:
#         list_id=list_id.json()
#         dict_id={}
#         for i in list_id:
#             dict_id[i["name"]]=i["id"]
#         sales_report["group"]=sales_report["group"].apply(lambda x:dict_id[x])
#         n=4
        
#     else:
#         n+=1
#         print(list_id.status_code)
#         sleep(5)


In [23]:
sales_report=sales_report.fillna(0)




rows_to_send = []


for index, row in sales_report.iterrows():
    json_sales_report = {
        "paid": row["payed"],
        "cartable_paid": row["kartabli_payed"],
        "paid_of_registration": row["paied_of_registration"],
        "paid_percent": row["paid_percent"],
        "number_of_registration": row["number_of_registration"],
        "number_of_cartable_registration": row["number_of_kartabli_registration"],
        "entries_count": row["entries_count"],
        "answered_count": row["answered_count"],
        "paid_coe": row["paid_coe"],
        "answered_percent": row["answered_percent"],
        "bought_percent": row["bought_percent"],
        "number_of_active_saller": row["creator"],
        "av_paid": row["av_paied"],
        "av_call_duration": row["av_call_duration"],
        "group_name": row["group"]
    }
    rows_to_send.append(json_sales_report)


# m = 0
# k=0
# for row in rows_to_send:
#     response = requests.post("http://192.168.24.134:8000/api/report/sales-add", json=row)
#     if response.status_code == 201:
#         print("done")
#         sleep(0.5)
#         k+=1
#         print(k)
#     else:
#         print(response.status_code)
#         m += 1
        
        
#         sleep(5)
#         if m==4:
#             break

In [24]:
sales_report

,group,entries_count,payed,kartabli_payed,paied_of_registration,number_of_registration,number_of_kartabli_registration,paid_percent,creator,av_paied,call_duration,av_call_duration,answered_count,paid_coe,answered_percent,bought_percent
0,تسویه شده ها,0.0,0.000,0.000,0.000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,تعلیقی ها,3.0,0.000,0.000,0.000,0.0,0.0,0.000000,1.0,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,گروه آموزشی_دورکار,0.0,0.000,0.000,0.000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,گروه ارتش سرخ (صبا افشاری),288.0,52.605,4.095,48.510,24.0,18.0,1.084416,10.0,5.260500e+06,5.933889,0.593389,136.0,30110.294118,0.472222,0.132353
4,گروه ارتش سرخ (مبینا ولدخانی),321.0,20.160,2.520,21.735,10.0,8.0,0.927536,6.0,3.360000e+06,0.302500,0.050417,84.0,30000.000000,0.261682,0.095238
5,گروه ارتش سرخ( آزاده غفاری),279.0,79.380,1.575,81.900,40.0,32.0,0.969231,11.0,7.216364e+06,5.930278,0.539116,136.0,11580.882353,0.487455,0.235294
6,گروه ارتش سرخ(مهری کرمی),360.0,61.740,5.670,54.495,28.0,23.0,1.132948,12.0,5.145000e+06,8.071667,0.672639,163.0,34785.276074,0.452778,0.141104
7,گروه ارتش سرخ_مینا آشفته دل,120.0,6.615,2.520,4.095,2.0,1.0,1.615385,2.0,3.307500e+06,0.111111,0.055556,27.0,93333.333333,0.225000,0.037037
8,گروه اوج_سعید بیدار,0.0,0.000,0.000,0.000,0.0,0.0,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,گروه اوج_سمیرا شهامتی,366.0,52.605,0.000,52.605,26.0,21.0,1.000000,9.0,5.845000e+06,7.116111,0.790679,131.0,0.000000,0.357923,0.160305


In [25]:
#ساخت فایل jsonنهایی
# data = {
#     "vosoli_hamkadeh": vosoli_hamkadeh,
#     "vosoli_kartabli_hamkadeh": vosoli_kartabli_hamkadeh,
#     "sabt_hamkadeh": sabt_hamkadeh,
#     "tedad_sabt_hamkadeh": tedad_sabt_hamkadeh,
#     "tedad_sabt_kartabli_hamkadeh": tedad_sabt_kartabli_hamkadeh,
#     "modat_mokalemehamkadeh": modat_mokalemehamkadeh,
#     "tedad_shomare_hamkadeh": tedad_shomare_hamkadeh,
#     "pasokhdad_hamkadeh": pasokhdad_hamkadeh,
#     "number_of_active_saler":number_of_active_saler
# }





# تبدیل داده‌ها به دیتافریم
# df = pd.concat(data.values(), axis=1, keys=data.keys())
# json_data = df.to_json(orient="index")



# data = json.loads(json_data)
# supervisor_json = json.dumps(data, ensure_ascii=False, indent=4)

#save
# with open("output.json", "w", encoding="utf-8") as fp:
#     fp.write(supervisor_json)